In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus
from astropy.wcs import WCS
from astropy.io import fits
from astropy.coordinates import SkyCoord

In [2]:
#Import file extract image, header and wcs
#Import reference specific to object name

In [ ]:
#Show image with reference stars plotted alongside

In [3]:
#Could maybe do diagnostics if new object since ignored ones should be the same for all
#Should at least develop the masked objects to remove from the positions


In [4]:
#Get positions tuple without masked objects

In [5]:
#Aperture optimisation
#Get r that corresponds to max S/N using argmax

In [6]:
#Read and calculate reference mags as necessary depending on the filter


In [7]:
#Calculate ZP using curve fit? or just mean or median and then take errors

In [ ]:
#Strap ZP to headers